In [ ]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

sys.path.append(os.path.abspath('..'))
from src.utils.seed import set_seed
from src.utils.config import Config
from src.preprocessing import get_data_loaders
from src.ensemble import StackingEnsemble
from src.evaluation import calculate_metrics

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
config = Config('../configs/config.yaml')
train_loader, val_loader, test_loader, input_dim = get_data_loaders(config)

X_train_list = []
y_train_list = []

for X, y in train_loader:
    X_train_list.append(X.numpy())
    y_train_list.append(y.numpy())

X_train = np.vstack(X_train_list)
y_train = np.concatenate(y_train_list)

print(f"Training data for ensemble: {X_train.shape}")

In [ ]:
ensemble = StackingEnsemble(config.ensemble, input_dim=input_dim)

print("Training stacking ensemble...")
ensemble.fit(X_train, y_train)
print("✓ Ensemble training complete!")

In [ ]:
X_test_list = []
y_test_list = []

for X, y in test_loader:
    X_test_list.append(X.numpy())
    y_test_list.append(y.numpy())

X_test = np.vstack(X_test_list)
y_test = np.concatenate(y_test_list)

y_pred_probs = ensemble.predict_proba(X_test)

test_metrics = calculate_metrics(y_test, y_pred_probs)

print("Ensemble Model Test Results:")
for metric, value in test_metrics.items():
    print(f"  {metric.upper()}: {value:.4f}")

In [ ]:
import joblib

os.makedirs('../reports/results/checkpoints', exist_ok=True)
joblib.dump(ensemble, '../reports/results/checkpoints/ensemble_model.joblib')

print("✓ Ensemble model saved!")